What is Auto-complete?

Sentence completion suggestions that we get when we starting our questions on Google.

Developing an Auto-complete system involves creating a Language Model (LM). 
LM assigns a probability to sequence(set) of words from a reference set like Wiki pages, articles etc.
Higher probability means higher score means the sequence would make more sense.

For eg. "I eat scrambled", now find x such that the sentence "I eat scrambled x" gets highest probability. If x='eggs', the sentence will be "I eat scrambled eggs".

Here we will use N-gram method for language modeling.

N-grams are also used in Machine translation and speech recognition.

"""
Our Steps:

1. Load data (paragraph format)
2. Preprocess the data:
    a) Read the US twitter data. Every tweet is separated by \n
    b) Split, clean and get individual nonempty tweets
    c) Tokenize into words and characters 
    d) Split data into train and test set (validation is not used for easy implementation)
    d) Handling OOV words :- 
        Get word freq
        Filter words with n+ freq
        Replace the other words with unk, unknown token
3. Develop n-gram Language model
    a) Count n grams
    b) Calculate conditional probability of next word with k-Smoothing
4. Calculate Perplexity(PP) metric - it is like entropy. Lower PP scores have better outputs
5. Bring every thing together and test an Auto-complete sentence

"""

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
from itertools import islice
from collections import Counter

nltk.data.path.append('.')

In [ ]:
def load_data(path, file):

    with open(path+file,'r') as f:
        data = f.read()[:10000] #load top 10000 characters
    
    return data

path = '/Users/apoorvasrivastava/Documents/ML/apoorva/NLP specialization/twitterData/final/en_US/'
file = 'en_US.twitter.txt'

data = load_data(path,file)

### Preprocess Data

In [ ]:
def get_tokenized_data(data):
    
    """
    1. split data and strip extra spaces and remove empty tweets
    2. tokenize each tweet into word tokens
    
    Input: string of text
    output: Tokenized word sets
    """
    
    def split_tweets(data):

        #split into different tweets
        sentences = data.split('\n')

        #strip extra spaces in front and back in each tweet
        tweets_strip = [line.strip() for line in sentences]

        #remove blank tweets
        nonempty_tweet = [tweet for tweet in tweets_strip if len(tweet)>0]
        
        return nonempty_tweet
    
        
    def tokenize(split_tweets):
    
        tok_tweets = []
        for tweet in split_tweets:

            #lowercase
            tweet_lower = tweet.lower()

            #tokenize
            tweet_tok = nltk.word_tokenize(tweet_lower)

            #append
            tok_tweets.append(tweet_tok)
   
        return tok_tweets
    
    
    split_tweets = split_tweets(data)
    tok_tweets = tokenize(split_tweets)
    
    return tok_tweets


In [ ]:
token_data = get_tokenized_data(data)

In [ ]:
def train_test_set(token_data):
    """
    divide text string into meaningful 90%, 10% split. 
    first 90% to train, next 10% to test
    """
    train_size = int(len(token_data)*0.8)
    
    train = token_data[:train_size]
    test = token_data[train_size:]
    
    return train, test

train, test = train_test_set(token_data)

In [ ]:
### Handling OOV words

In [ ]:
def hand_OOV_words(token_data,threshold):
    
    """
    1. calculate word frequency
    2. create a closed vocab by only selecting words>n threshold
    2.1 rename all the other words as <unk>
    
    Input: tokenized data, threshold cutoff
    Output: closed vocabulary, all replaced tokens also
    """
    
    def word_freq(token_data):
        
        all_token_flat = []
        #flatten the list
        for element in token_data:
            for i in element:
                all_token_flat.append(i)
                
        word_freq = Counter(all_token_flat)
    
        return word_freq
    
    
    word_freq = word_freq(token_data)
    
    
    def closed_vocab(word_freq, threshold):
        
        #sort dict by val reversed
        sort_word_freq = dict(sorted(word_freq.items(), key=lambda x:x[1], reverse=True))
        
        #select only words higher than 1 freq
        cutoff_lower_data = dict(filter(lambda x: x[1]>1, sort_word_freq.items()))
        
        closed_vocab = list(cutoff_lower_data.keys())
        
        return closed_vocab, sort_word_freq
    
    
    closed_vocab, sort_word_dict = closed_vocab(word_freq, threshold)
    
    
    def replace_unknown(sort_word_dict,unknown='<unk>'):

        updated_tokens=[]
        
        for key,val in sort_word_dict.items():
            if val<2:
                updated_tokens.append(unknown)
            else:
                updated_tokens.append(key)
        
        return updated_tokens
    
    updated_tokens = replace_unknown(word_freq)
    
    
    return closed_vocab, updated_tokens


closed_vocab,updated_tokens=hand_OOV_words(token_data,threshold=5)    

In [ ]:
closed_vocab

### Develop n-gram language model

In [123]:
def count_ngram(sentences_token, n):
    
    """
    1. for every sentence, initialize m window of size n
    2. until m reaches m=length-n 
    3. keep appending to ngram list
    
    Input: List of list tokenized sentences 
    Output: list of ngrams across sentences
    """
    
    ngram=[]
    
    for sentence in sentences_token:
        m=0
        while m+n <= len(sentence):
            ngram.append(sentence[m:m+n])
            m+=1
    return ngram

In [124]:
sentences_token=[['on',
 'do',
 'is',
 'be','ll','mk'],['opn',
 'dpo',
 'ips',
 'bpe','lo']]

a = count_ngram(sentences_token, n=3)
print(a)

[['on', 'do', 'is'], ['do', 'is', 'be'], ['is', 'be', 'll'], ['be', 'll', 'mk'], ['opn', 'dpo', 'ips'], ['dpo', 'ips', 'bpe'], ['ips', 'bpe', 'lo']]
